# Relation Word2Vec with synonymy or hypernymy
The implementation os skip gram model might not be the exact one presented in the paper but I tried to do it similarly.

Reference for the skip gram model according to a Github implementation:
https://github.com/n0obcoder/Skip-Gram_Model-TensorFlow/blob/master/model.py

I added relation and modified the loss function.

You just neeed to run the notebook inside colab or your own environmnet. I suggest colab as every library is already included there so no need for extra work.

In [ ]:
# Some text data from wikipedia
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile
dataset_folder_path = 'content'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'
class DLProgress(tqdm):
    last_block = 0
    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num
if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)
if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)

Text8 Dataset: 31.4MB [00:34, 905kB/s]                            


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
relation_matrix_path = "/content/gdrive/MyDrive/GloveEmbeddings/relationMatrix_improved.npy"

In [ ]:
# Upload the data to google cloud in case the drag-drop upload is not working.
from google.colab import files
dataset_file_dict = files.upload()

Saving corpus_graph.gpickle to corpus_graph.gpickle


## Word2Vec with Synonymy or Hypernymy Relations

## Import and Preleminaries

In [ ]:
import os
import tensorflow as tf
import networkx as nx
import numpy as np
import zipfile
from collections import Counter
import random
FILE_PATH = "text8.zip"

In [ ]:
import os

tf.test.is_gpu_available()
DATA_SOURCE = 'gensim' # or 'toy'
# DATA_SOURCE = 'toy'
MODEL_ID = DATA_SOURCE #'toy'# 'gensim'
DISPLAY_BATCH_LOSS = True
DISPLAY_EVERY_N_BATCH = 1000
SAVE_EVERY_N_EPOCH = 1
BATCH_SIZE = 1024*16
NUM_EPOCHS = 1
CONTEXT_SIZE = 3
FRACTION_DATA = 1
SUBSAMPLING = True
SAMPLING_RATE = 0.001
NEGATIVE_SAMPLES = 20 # set it to 0 if you don't want to use negative samplings
EMBEDDING_DIM = 100
LR = 0.001
if FRACTION_DATA == 1:
    TEST_WORDS = ['india', 'computer', 'gold', 'football', 'cars', 'war', 'apple', 'music', 'helicopter']
    TEST_WORDS_VIZ = ['india', 'asia', 'guitar', 'piano', 'album', 'music', 'war', 'soldiers', 'helicopter']
else:
    TEST_WORDS = ['human', 'boy', 'office', 'woman']
    TEST_WORDS_VIZ = TEST_WORDS

PREPROCESSED_DATA_DIR = os.path.join(MODEL_ID, 'preprocessed_data')
PREPROCESSED_DATA_PATH = os.path.join(PREPROCESSED_DATA_DIR, 'preprocessed_data_' + MODEL_ID + '_' + str(FRACTION_DATA) + '.pickle')
SUMMARY_DIR = os.path.join(MODEL_ID, 'summary')
MODEL_DIR = os.path.join(MODEL_ID, 'models')
DATA_DICT_PATH = os.path.join(MODEL_ID, 'data_dict.pickle')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


### Helper Function

In [ ]:
import numpy as np
import sys

def q():
    sys.exit()

def count_parameters(model):
    num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return num_parameters/1e6 # in terms of millions

# TEST
def nearest_word(inp, emb, top = 5, debug = False):
    euclidean_dis = np.linalg.norm(inp - emb, axis = 1)
    emb_ranking = np.argsort(euclidean_dis)
    emb_ranking_distances = euclidean_dis[emb_ranking[:top]]
    emb_ranking_top = emb_ranking[:top]
    euclidean_dis_top = euclidean_dis[emb_ranking_top]
    if debug:
        print('euclidean_dis: ', euclidean_dis)
        print('emb_ranking: ', emb_ranking)
        print(f'top {top} embeddings are: {emb_ranking[:top]} with respective distances\n {euclidean_dis_top}')
    return emb_ranking_top, euclidean_dis_top

In [ ]:
import numpy as np
import pickle, pdb
import tensorflow as tf

def print_nearest_words_tf(model, test_words, word_to_ix, ix_to_word, top = 5):
    emb_matrix = model.embeddings_input(np.array(range(len(word_to_ix))))
    print('type(emb_matrix): ', type(emb_matrix))
    nearest_words_dict = {}
    print('==============================================')
    for t_w in test_words:
        inp_emb = emb_matrix[word_to_ix[t_w], :]
        emb_ranking_top, _ = nearest_word(inp_emb, emb_matrix, top = top+1)
        print(t_w.ljust(10), ' | ', ', '.join([ix_to_word[i] for i in emb_ranking_top[1:]]))
    return nearest_words_dict

### Data Loader

In [ ]:
from __future__ import print_function

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

import numpy as np
import networkx as nx
import os, glob, cv2, sys, pdb, random, time
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.utils import Sequence

lem = WordNetLemmatizer()

class original_wordGraph2vec_dataset(Sequence):
    def __init__(self, DATA_SOURCE, BATCH_SIZE, CONTEXT_SIZE, FRACTION_DATA, SUBSAMPLING, SAMPLING_RATE):
        print("Parsing text and loading training data...")
        vocab, word_to_ix, ix_to_word, training_data = self.load_data(DATA_SOURCE, CONTEXT_SIZE, FRACTION_DATA, SUBSAMPLING, SAMPLING_RATE)
        self.batch_size = BATCH_SIZE
        self.vocab = vocab
        self.word_to_ix = word_to_ix
        self.ix_to_word = ix_to_word
        # training_data is a list of list of 2 indices
        self.data = np.array(training_data, dtype = np.uint64)
        self.indexes = np.arange(self.data.shape[0])

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        x = self.data[indexes, 0]
        y = self.data[indexes, 1]
        return x, y

    def __len__(self):
        return int(len(self.data)/self.batch_size)

    def gather_training_data(self, split_text, word_to_ix, ix_to_word, context_size):
        training_data = []
        all_vocab_indices = list(range(len(word_to_ix)))
        #for each sentence
        print('preparing training data (x, y, graphx graphy)...')
        for sentence in tqdm(split_text):
            indices = [word_to_ix[word] for word in sentence]
            context_words = []
            #for each word treated as center word
            for center_word_pos in range(len(indices)):
                #for each window  position
                for w in range(-context_size, context_size+1):
                    context_word_pos = center_word_pos + w
                    #make sure we dont jump out of the sentence
                    if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                        continue
                    context_word_idx = indices[context_word_pos]
                    center_word_idx  = indices[center_word_pos]
                    if center_word_idx == context_word_idx: # same words might be present in the close vicinity of each other. we want to avoid such cases
                        continue
                    training_data.append([center_word_idx, context_word_idx])
        return training_data

    def load_data(self, data_source, context_size, fraction_data, subsampling, sampling_rate):
        stop_words = set(stopwords.words('english'))
        if data_source == 'toy':
            sents = [
                    'word1 word2 word3 word4 word5',
                    'word6 word7 word8 word9 word10',
                    'word11 word12 word13 word14 word15'
                    ]
            # sents = ['word6 word7 word8 word9 word10', 'word1 word1 word1 word2 word2 word3 word4 word5', 'word11 word12 word13 word14 word15']
        elif data_source == 'gensim':
            import gensim.downloader as api
            dataset = api.load("text8")
            data = [d for d in dataset][:int(fraction_data*len([d_ for d_ in dataset]))]
            print(f'fraction of data taken: {fraction_data}/1')
            sents = []
            print('forming sentences by joining tokenized words...')
            for d in tqdm(data):
                sents.append(' '.join(d))
        sent_list_tokenized = [word_tokenize(s) for s in sents]
        print('len(sent_list_tokenized): ', len(sent_list_tokenized))
        # remove the stopwords
        sent_list_tokenized_filtered = []
        print('lemmatizing and removing stopwords...')
        for s in tqdm(sent_list_tokenized):
            sent_list_tokenized_filtered.append([w for w in s if w not in stop_words])
        sent_list_tokenized_filtered, vocab, word_to_ix, ix_to_word = self.gather_word_freqs(sent_list_tokenized_filtered, subsampling, sampling_rate)
        training_data = self.gather_training_data(sent_list_tokenized_filtered, word_to_ix, ix_to_word, context_size)
        return vocab, word_to_ix, ix_to_word, training_data

    def gather_word_freqs(self, split_text, subsampling, sampling_rate): #here split_text is sent_list
        vocab = {}
        ix_to_word = {}
        word_to_ix = {}
        total = 0.0
        print('building vocab...')
        for word_tokens in tqdm(split_text):
            for word in word_tokens: #for every word in the word list(split_text), which might occur multiple times
                if word not in vocab: #only new words allowed
                    vocab[word] = 0
                    ix_to_word[len(word_to_ix)] = word
                    word_to_ix[word] = len(word_to_ix)
                vocab[word] += 1.0 #count of the word stored in a dict
                total += 1.0 #total number of words in the word_list(split_text)
        print('\nsubsampling: ', subsampling)
        if subsampling:
            print('performing subsampling...')
            for sent in tqdm(split_text):
                word_tokens = sent
                # print('word_tokens: ', word_tokens)
                # print('len(word_tokens): ', len(word_tokens), '\n')
                for i , word in enumerate(word_tokens):
                    # print(i, word_tokens[i])
                    frac = vocab[word]/total
                    prob = 1 - np.sqrt(sampling_rate/frac)
                    sampling = np.random.sample()
                    #print(sampling, prob)
                    if (sampling < prob):
                        # print('freq: ', vocab[word_tokens[i]])
                        del word_tokens[i]
                        i -= 1
        return split_text, vocab, word_to_ix, ix_to_word

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Relation W2Vec

In [ ]:
import numpy as np
def get_synonym_relation_matrix(path='/content/relationMatrix.npy'):
    R = np.load(path)
    return R[:, 1, :]

def get_hypernym_relation_matrix(path='/content/relationMatrix.npy'):
    R = np.load(path)
    return R[:, 0, :]

In [ ]:
# R = get_synonym_relation_matrix(relation_matrix_path)
R = get_hypernym_relation_matrix(relation_matrix_path)

In [ ]:
# Get relation for a specific word in a batch of words
def get_relations_between_words(R, target_idx, context_idx):
    rel = np.zeros((target_idx.shape[0], 1))
    t = R.shape[0]
    c = R.shape[1]
    for i, target in enumerate(target_idx):
        if target < t and context_idx[i] < c:
            rel[i] = R[target, context_idx[i]]
    return rel

In [ ]:
print(R.shape)

(5000, 5000)


In [ ]:
import numpy as np
x = np.array([1, 2, 3, 4])
y = np.array([5, 6, 7, 8])
# Test multiplication in both axis
z = np.prod((x, y), axis=0)
print(z)

[ 5 12 21 32]


## Model and Loss function

In [ ]:
import tensorflow as tf

class relations_word2vec_tf(tf.keras.Model):

    def __init__(self, embedding_size, vocab_size, noise_dist = None, negative_samples = 0):
        super(relations_word2vec_tf, self).__init__()
        self.embeddings_input   = tf.keras.layers.Embedding(vocab_size, embedding_size, embeddings_initializer='uniform', mask_zero=False, name="w2v_embedding")
        self.embeddings_context = tf.keras.layers.Embedding(vocab_size, embedding_size, embeddings_initializer='uniform', mask_zero=False)
        self.vocab_size = vocab_size
        self.negative_samples = negative_samples
        self.noise_dist = noise_dist

    def call(self, input_word, context_word, relations):
        debug = not True
        if debug:print('input_word  : ', input_word.shape)
        # print('input_word  : ', input_word.shape)
        if debug:print('context_word: ', context_word.shape)
        # print('context_word: ', context_word.shape)
        ##### computing out loss #####
        emb_input = self.embeddings_input(input_word)
        if debug:print('emb_input: ', emb_input.shape)
        emb_context = self.embeddings_context(context_word)
        if debug:print('emb_context: ', emb_context.shape)

        # Relation synonymy or hypernymy relations
        emb_product = tf.keras.layers.dot([emb_input, emb_context], axes = (1, 1))
        # Apply the knowledge-regularized method
        emb_product = emb_product + tf.math.reduce_prod([relations, emb_product], axis=0)

        if debug:print('emb_product.shape: ', emb_product.shape)
        out_loss = tf.squeeze(tf.math.log_sigmoid(emb_product), axis = 1)
        if debug:print('out_loss.shape: ', out_loss.shape)
        return tf.reduce_mean(tf.math.negative(out_loss))
    def this(self):
        print('testing this!')

Download embedding into files

In [ ]:
import io
def download_embedding_file(word2vec, vocab, title=''):
    weights = word2vec.get_layer('w2v_embedding').get_weights()[0]

    # Create and save the vectors and metadata file.
    out_v = io.open(f'vectors_{title}.tsv', 'w', encoding='utf-8')
    embeddings_file = io.open(f'embeddings_{title}.tsv', 'w', encoding='utf-8')
    out_m = io.open(f'metadata_{title}.tsv', 'w', encoding='utf-8')

    for index, word in enumerate(vocab):
        if index == 0:
            continue  # skip 0, it's padding.
        vec = weights[index]
        embeddings_file.write(word + '\t' + '\t'.join([str(x) for x in vec]) + "\n")
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        out_m.write(word + "\n")
    out_v.close()
    out_m.close()
    embeddings_file.close()
    try:
        from google.colab import files
        files.download(f'vectors_{title}.tsv')
        files.download(f'embeddings_{title}.tsv')
        files.download(f'metadata_{title}.tsv')
    except Exception as e:
        print('Error while downloading the file: ', e)

### Training the Model

In [ ]:
from __future__ import print_function
from tqdm import tqdm
# from tqdm import tqdm_gui
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import sys, pdb, os, shutil, pickle
from pprint import pprint
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
print('gpus: ', gpus)
# tf.config.experimental.set_memory_growth(gpus[0], True)
# remove MODEL_DIR if it exists
if os.path.exists(MODEL_DIR):
    shutil.rmtree(MODEL_DIR)
# create fresh MODEL_DIR
os.makedirs(MODEL_DIR)
write_summary = True
if write_summary:
    # SUMMARY_DIR is the path of the directory where the tensorboard SummaryWriter files are written
    # the directory is removed, if it already exists
    if os.path.exists(SUMMARY_DIR):
        shutil.rmtree(SUMMARY_DIR)
    # os.makedirs(SUMMARY_DIR)
    train_summary_writer = tf.summary.create_file_writer(SUMMARY_DIR)
    summary_counter = 0

# make training data and dataloader
if not os.path.exists(PREPROCESSED_DATA_PATH):
    train_dataset = original_wordGraph2vec_dataset(DATA_SOURCE, BATCH_SIZE, CONTEXT_SIZE, FRACTION_DATA, SUBSAMPLING, SAMPLING_RATE)
    if not os.path.exists(PREPROCESSED_DATA_DIR):
        os.makedirs(PREPROCESSED_DATA_DIR)
    # pickle dump
    print('\ndumping pickle...')
    outfile = open(PREPROCESSED_DATA_PATH,'wb')
    pickle.dump(train_dataset, outfile)
    outfile.close()
    print('pickle dumped\n')
else:
    # pickle load
    print('\nloading pickle...')
    infile = open(PREPROCESSED_DATA_PATH,'rb')
    train_dataset = pickle.load(infile)
    train_dataset.batch_size = BATCH_SIZE
    infile.close()
    print('pickle loaded\n')
print('len(train_dataset): ', len(train_dataset))
vocab = train_dataset.vocab
word_to_ix = train_dataset.word_to_ix
ix_to_word = train_dataset.ix_to_word

# saving some data information in a pickle file, to be used later while inference/testing
# dump data_dict pickle
# OVERRITING THE OLDER PICKLE FILE
print('\ndumping data_dict pickle...')
outfile = open(DATA_DICT_PATH,'wb')
data_dict = {
    'vocab': vocab,
    'word_to_ix': word_to_ix,
    'ix_to_word': ix_to_word
    }
pickle.dump(data_dict, outfile)
outfile.close()
print('pickle dumped\n')
print('len(vocab): ', len(vocab), '\n')

# make noise distribution to sample negative examples from
word_freqs = np.array(list(vocab.values()))
unigram_dist = word_freqs/sum(word_freqs)
noise_dist = unigram_dist**(0.75)/np.sum(unigram_dist**(0.75))

# Vanilla Skip-Grams : NEGATIVE_SAMPLES = 0
NEGATIVE_SAMPLES = 0
model = relations_word2vec_tf(EMBEDDING_DIM, len(vocab), noise_dist, NEGATIVE_SAMPLES)
optimizer = tf.keras.optimizers.Adam(LR)
print('TRAINING...')
for epoch in tqdm(range(NUM_EPOCHS)):
    print('\n===== EPOCH {}/{} ====='.format(epoch + 1, NUM_EPOCHS))
    # For orginial wg2vec model
    for batch_idx, (x, y) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            relations = get_relations_between_words(R, x, y)
            loss_value = model(x, y, relations)
            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
        print('batch# ' + str(batch_idx+1).zfill(len(str(len(train_dataset)))) + '/' + str(len(train_dataset)) + ' | Loss: ' + str(round(loss_value.numpy(), 5)), end = '\r')
        if batch_idx%DISPLAY_EVERY_N_BATCH == 0 and DISPLAY_BATCH_LOSS:
            print(f'Batch: {batch_idx+1}/{len(train_dataset)}, Loss: {loss_value}')
            # show 5 closest words to some test words
            print_nearest_words_tf(model, TEST_WORDS, word_to_ix, ix_to_word, top = 5)
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', loss_value.numpy(), step=summary_counter)
            summary_counter += 1
    # write embeddings every SAVE_EVERY_N_EPOCH epoch
    if epoch%SAVE_EVERY_N_EPOCH == 0:
        model.save_weights('{}/model{}'.format(MODEL_DIR, epoch))
download_embedding_file(model, vocab, 'original_graph_wg2v')

gpus:  []
Parsing text and loading training data...
[==================================================] 100.0% 31.6/31.6MB downloaded
fraction of data taken: 1/1
forming sentences by joining tokenized words...


100%|██████████| 1701/1701 [00:00<00:00, 3957.82it/s]


len(sent_list_tokenized):  1701
lemmatizing and removing stopwords...


100%|██████████| 1701/1701 [00:02<00:00, 619.88it/s]


building vocab...


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>